In [ ]:
import faiss
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer
import torch

# ----------------------------
# Load model & indexes
# ----------------------------

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

# Load index (word-based example)
index = faiss.read_index('../data/processed/faiss_word_sample.index')
with open('../data/processed/word_metadata_sample.pkl', 'rb') as f:
    metadata = pickle.load(f)

# ----------------------------
# Define retriever
# ----------------------------

def retrieve_top_k(query, k=5):
    query_emb = model.encode([query]).astype('float32')
    D, I = index.search(query_emb, k)
    results = []
    for i in I[0]:
        result = {
            "chunk": word_chunks_sample[i],  # or recursive_chunks_sample
            "metadata": metadata[i]
        }
        results.append(result)
    return results

# ----------------------------
# Test retrieval
# ----------------------------

query = "Why are people unhappy with credit card charges?"
results = retrieve_top_k(query, k=5)

print("\n🔍 Top retrieved chunks:")
for res in results:
    print("-", res['chunk'][:200], "...")
